In [1]:
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
import biogeme.messaging as message
from biogeme import expressions as ex
import pandas as pd
import numpy as np
import xlsxwriter

# Calibration
## Estimation of the model parameters
quetzal_germany is being estimated using [PandasBiogeme](https://biogeme.epfl.ch/). This notebook estimates calibration parameters for the model's utility functions.
- Documentation and reference: [Bierlaire, M. (2020). A short introduction to PandasBiogeme. Technical report TRANSP-OR 200605. Transport and Mobility Laboratory, ENAC, EPFL.](https://transp-or.epfl.ch/documents/technicalReports/Bier20.pdf)
- Tutorial: https://www.youtube.com/watch?v=OiM94B8WayA

### Model formulation
The model consists of systematic utility functions, one for each mode. They comprise an alternaive-specific constant (ASC), a distance-dependent part with travel time and cost summarised as generalised cost (GC), and a cost damping function F

$V_i = ASC + F(GC(T, C), b_{gc_i})$

Index i marks the demand group. I = {'commuting' (1), 'education' (2), 'shopping/medical' (3), 'business' (4), 'private' (6)}

Note: The cost variable already includes subscriptions

In [2]:
input_path = '../input/'
model_path = '../model/'

### Prepare the database

In [3]:
df = pd.read_csv(input_path + 'transport_demand/calibration_inter-cellular_trips_MiD2017.csv')
df = df[['cost_rail_short', 'cost_rail_long', 'cost_car', 'cost_coach', 'cost_bus', 'cost_walk', 'cost_air',
         'time_rail_short', 'time_rail_long', 'time_car', 'time_coach', 'time_bus', 'time_walk', 'time_air',
         'cost_rail', 'cost_road', 'time_rail', 'time_road',
#         'accessibility_rail', 'accessibility_car', 'accessibility_coach', 'accessibility_bus',
#         'accessibility_walk', 'accessibility_air',
         'mode_model', 'purpose_vp', 'car_avail', 'length']]
df.columns = ['C_RAIL_S', 'C_RAIL_L', 'C_CAR', 'C_COACH', 'C_BUS', 'C_NON_MOTOR', 'C_AIR',
              'T_RAIL_S', 'T_RAIL_L', 'T_CAR', 'T_COACH', 'T_BUS', 'T_NON_MOTOR', 'T_AIR',
              'C_RAIL', 'C_ROAD', 'T_RAIL', 'T_ROAD',
#              'AC_RAIL', 'AC_CAR', 'AC_COACH', 'AC_BUS', 'AC_NON_MOTOR', 'AC_AIR',
              'MODE', 'PURPOSE', 'CAR_AV', 'DIST']

In [4]:
inf = 1000
df = df.replace({np.inf:inf})

In [5]:
df.describe()

,C_RAIL_S,C_RAIL_L,C_CAR,C_COACH,C_BUS,C_NON_MOTOR,C_AIR,T_RAIL_S,T_RAIL_L,T_CAR,...,T_NON_MOTOR,T_AIR,C_RAIL,C_ROAD,T_RAIL,T_ROAD,MODE,PURPOSE,CAR_AV,DIST
count,95284.000000,95284.000000,95284.000000,95284.000000,95284.000000,95284.0,95284.000000,95284.000000,95284.000000,95284.000000,...,95284.000000,95284.000000,95284.000000,95284.000000,95284.000000,95284.000000,95284.000000,95284.000000,95284.000000,95284.000000
mean,15.014242,82.547716,10.857505,7.741327,2.782476,0.0,949.784468,140.415455,578.886315,82.605621,...,453.160793,961.785732,17.024913,5.362768,110.557676,213.499049,5.553136,3.654381,0.964412,99.105247
std,13.099687,56.580329,12.741625,7.034933,1.156256,0.0,207.662853,185.499968,422.730710,88.539893,...,450.080379,158.845560,17.326776,6.033399,98.407942,255.945144,1.352147,2.059509,0.189297,140.837481
min,0.000000,19.000000,0.066667,5.000000,0.000000,0.0,35.000000,29.783333,39.933333,10.000000,...,2.660824,138.166667,0.000000,0.000000,13.000000,11.783333,1.000000,1.000000,0.000000,0.000000
25%,6.291333,19.487458,4.252006,5.000000,2.740000,0.0,1000.000000,56.283333,129.700000,36.183333,...,76.333333,1000.000000,6.916239,2.740000,56.183333,63.133333,6.000000,1.000000,1.000000,30.207000
50%,11.071095,71.980586,7.282322,5.000000,2.740000,0.0,1000.000000,75.000000,467.500000,50.916667,...,120.150000,1000.000000,11.711845,4.000000,74.016667,109.550000,6.000000,3.000000,1.000000,45.874000
75%,18.343221,139.000000,11.079886,5.000000,4.000000,0.0,1000.000000,123.300000,1000.000000,78.100000,...,1000.000000,1000.000000,19.424456,5.000000,119.508333,234.016667,6.000000,6.000000,1.000000,83.463000
max,42.000000,139.000000,113.828808,56.306642,4.000000,0.0,1000.000000,1000.000000,1000.000000,646.300000,...,1000.000000,1000.000000,139.000000,47.074163,1000.000000,1000.000000,7.000000,6.000000,9.000000,994.817000


In [6]:
# Scale time to hours
df['T_RAIL_Sc'] = df['T_RAIL'] / 60
df['T_RAIL_S_S'] = df['T_RAIL_S'] / 60
df['T_RAIL_L_S'] = df['T_RAIL_L'] / 60
df['T_CAR_S'] = df['T_CAR'] / 60
df['T_COACH_S'] = df['T_COACH'] / 60
df['T_BUS_S'] = df['T_BUS'] / 60
df['T_ROAD_S'] = df['T_ROAD'] / 60
df['T_AIR_S'] = df['T_AIR'] / 60
df['T_NON_MOTOR_S'] = df['T_NON_MOTOR'] / 60

In [7]:
# Make car availability binary
df['CAR_AV'] = df['CAR_AV'].replace({9:0})

In [8]:
# Add PT availabilities
df['RAIL_AV'] = (df['T_RAIL']!=inf).astype(int)
df['RAIL_S_AV'] = (df['T_RAIL_S']!=inf).astype(int)
df['RAIL_L_AV'] = (df['T_RAIL_L']!=inf).astype(int)
df['COACH_AV'] = (df['T_COACH']!=inf).astype(int)
df['BUS_AV'] = (df['T_BUS']!=inf).astype(int)
df['ROAD_AV'] = (df['T_ROAD']!=inf).astype(int)
df['AIR_AV'] = (df['T_AIR']!=inf).astype(int)
df['NON_MOTOR_AV'] = (df['C_NON_MOTOR']!=inf).astype(int)

In [9]:
# Merge rail and road PT
df['MODE'] = df['MODE'].replace({2:1, 3:4})
len(df.loc[df['MODE'].isin([2,3])])

0

In [10]:
# Remove trips where mode is car but the car availability is zero
# because it irritates the MLE algorithm
mask = ((df['MODE']==6) & (df['CAR_AV']==0))
print('Share of car trips dropped: {}'.format(
    len(df.loc[mask])/len(df.loc[df['MODE']==6])))
df = df.loc[~mask]

Share of car trips dropped: 0.014151397104794882


In [11]:
# Remove trips where cost are infinity
# Share of drops per mode must be equal, otherwise the calibration is skewed
# Ignore air trips because this dataset has too few observations anyways
max_drop_ratio = 0
lengths = []
for mode, col in zip([1, 4], ['RAIL_AV', 'ROAD_AV']):
    drops = df.loc[((df['MODE']==mode) & ~(df[col]))].index
    lengths = lengths + list(df.loc[drops, 'DIST'])
    if len(drops) > 0: print('mode ' + str(mode) + ': ' + str(len(drops)) + ' drops')
    max_drop_ratio = max(len(drops) / len(df.loc[df['MODE']==mode]), max_drop_ratio)
print('max_drop_ratio: ' + str(max_drop_ratio))
if max_drop_ratio > 0:
    print('Drop trips length mean: {}; min: {}; max: {}'.format(
        sum(lengths)/len(lengths), min(lengths), max(lengths)))

max_drop_ratio: 0.0


In [12]:
# Drop trips with infinite cost
if max_drop_ratio > 0:
    df = df.loc[~((df['MODE']==2) & (df['C_RAIL_L']==inf))]
    # Don't drop air trips because there are only a few
    for m in [1, 3, 4, 6]:
        n_drops = int(max_drop_ratio * len(df.loc[df['MODE']==m]))
        df = df.drop(df.loc[(df['MODE']==m) & (df['DIST']<max(lengths)) &
                            (df['DIST']>min(lengths))].sample(n_drops).index)
        print('mode ' + str(m) + ': ' + str(n_drops) + ' drops')
    print('New number of observations: ' + str(len(df.index)))

### Model variables
All columns are variables. DefineVariable creates a new column in the database.

#### Cost damping

Many modelling studies have shown that cost damping is required in order to flatten the tail of time/cost elasticities, i.e. decrease the impact of long distances on choice results to prevent from overestimation of time/cost parameters. Cost damping represents the property of decreasing marginal utility. It is commonly approached with Box-Cox transformations of generalised cost (usually defined as the sum of travel time and travel expenditures divided by the value of time). Daly (2010) proposes a hybrid function as a sum of the linear term and the narural logarithm of the same. However, the linear term still dominates cost on long distances. Rich (2015), main developer of the Danish National Transport Model, proposes a more complex spline function which successfully manages cost damping and even outperforms the Box-Cox transformation in terms of stability of elasticities.

In [13]:
# Using the hybrid linear-logarithmic version requires a new column
# for the logarithm of time in minutes
'''df['T_RAIL_S'] = np.log(df['T_RAIL'])
df['T_CAR_S'] = np.log(df['T_CAR'])
df['T_COACH_S'] = np.log(df['T_COACH'])
df['T_BUS_S'] = np.log(df['T_BUS'])
df['T_AIR_S'] = np.log(df['T_AIR'])
df['T_NON_MOTOR_S'] = np.log(df['T_NON_MOTOR'])'''
print('This functional form performes poorly')

This functional form performes poorly


#### Generalised cost

Both, travel time and monetary cost should be affected by cost damping measures. It is logical to define a generalised cost term `GC` with dimension of time units. This requires definition or estimation of values of time, in order to rescale monetary units, for all segments. Usually, the value of time (VoT) is distance-dependent. In the case of Germany, VoT can be taken from research undertaken within the Federal Government's transport study "Bundesverkehrswegeplan 2030": Axhausen et al. 2015. Ermittlung von Bewertungsansätzen für Reisezeiten und Zuverlässigkeit auf der Basis eines Modells für modale Verlagerungen im nicht-gewerblichen und gewerblichen Personenverkehr für die Bundesverkehrswegeplanung

In [13]:
# VoT from literature, distance-dependent, see cal19
VoT = pd.read_csv(input_path + 'vot.csv', header=[0,1], index_col=0)
VoT.sample(2)

,root,Fz1,Fz2,Fz3,Fz4,Fz6,root,Fz1,Fz2,Fz3,...,Fz2,Fz3,Fz4,Fz6,root,Fz1,Fz2,Fz3,Fz4,Fz6
,all,all,all,all,all,all,PT,PT,PT,PT,...,air,air,air,air,car,car,car,car,car,car
47,9.3450,9.1270,7.809,11.076,10.5960,8.0640,7.3890,6.4170,8.319,10.8870,...,19.160,19.160,24.4700,15.8300,9.4910,9.2390,6.4410,12.6950,10.666,8.0640
672,20.2764,19.3988,9.180,11.900,26.4684,15.2952,15.2728,11.7052,13.480,16.1352,...,69.604,69.604,164.8516,39.2768,20.0576,19.0356,11.9768,25.3848,22.158,15.2952


In [14]:
# Make distance integer
df['DIST'] = df['DIST'].apply(int)

In [15]:
# Generate generalised cost
VoT = VoT.to_dict()
df['GC_RAIL'] = df['T_RAIL_Sc'] + [c / VoT[('Fz'+str(p), 'PT')][d]
                                   for c,d,p in zip(df['C_RAIL'], df['DIST'], df['PURPOSE'])]
df['GC_RAIL_S'] = df['T_RAIL_S_S'] + [c / VoT[('Fz'+str(p), 'PT')][d]
                                      for c,d,p in zip(df['C_RAIL_S'], df['DIST'], df['PURPOSE'])]
df['GC_RAIL_L'] = df['T_RAIL_L_S'] + [c / VoT[('Fz'+str(p), 'PT')][d]
                                      for c,d,p in zip(df['C_RAIL_L'], df['DIST'], df['PURPOSE'])]
df['GC_COACH'] = df['T_COACH_S'] + [c / VoT[('Fz'+str(p), 'PT')][d]
                                    for c,d,p in zip(df['C_COACH'], df['DIST'], df['PURPOSE'])]
df['GC_BUS'] = df['T_BUS_S'] + [c / VoT[('Fz'+str(p), 'PT')][d]
                                for c,d,p in zip(df['C_BUS'], df['DIST'], df['PURPOSE'])]
df['GC_ROAD'] = df['T_ROAD_S'] + [c / VoT[('Fz'+str(p), 'PT')][d]
                                  for c,d,p in zip(df['C_ROAD'], df['DIST'], df['PURPOSE'])]
df['GC_AIR'] = df['T_AIR_S'] + [c / VoT[('Fz'+str(p), 'air')][d]
                                for c,d,p in zip(df['C_AIR'], df['DIST'], df['PURPOSE'])]
df['GC_CAR'] = df['T_CAR_S'] + [c / VoT[('Fz'+str(p), 'car')][d]
                                for c,d,p in zip(df['C_CAR'], df['DIST'], df['PURPOSE'])]
df['GC_NON_MOTOR'] = df['T_NON_MOTOR_S']

In [16]:
# Create the initial database and make columns global variables
database = db.Database('MiD2017', df.copy())
globals().update(database.variables)
database.getSampleSize()

94107

### Estimation parameters

In [17]:
asc_rail = ex.Beta('asc_rail', 0, None, None, 0)
asc_rail_s = ex.Beta('asc_rail_s', 0, None, None, 0)
asc_rail_l = ex.Beta('asc_rail_l', 0, None, None, 0)
asc_coach = ex.Beta('asc_coach', 0, None, None, 0)
asc_bus = ex.Beta('asc_bus', 0, None, None, 0)
asc_road = ex.Beta('asc_road', 0, None, None, 0)
asc_air = ex.Beta('asc_air', 0, None, None, 0)
asc_car = ex.Beta('asc_car', 0, None, None, 1)
asc_non_motor = ex.Beta('asc_non_motor', 0, None, None, 0)

In [19]:
b_t = ex.Beta('b_t', 0, None, None, 0)
b_c = ex.Beta('b_c', 0, None, None, 0)
b_ac = ex.Beta('b_ac', 0, None, None, 0)

In [20]:
# non-linear time component
b_t2 = ex.Beta('b_t2', 0, None, None, 0)

In [18]:
# generalised cost function parameter
b_gc = ex.Beta('b_gc', 0, None, None, 0)

In [19]:
# Parameters for the nested logit structure
mu_pt = ex.Beta('mu_pt', 1, 1, 10, 0)

### Utility functions

In [23]:
'''# Aggregated formulation
V_RAIL = asc_rail + b_t * T_RAIL_S + b_c * C_RAIL + b_ac * AC_RAIL
V_COACH = asc_coach + b_t * T_COACH_S + b_c * C_COACH + b_ac * AC_COACH
V_BUS = asc_bus + b_t * T_BUS_S + b_c * C_BUS + b_ac * AC_BUS
V_AIR = asc_air + b_t * T_AIR_S + b_c * C_AIR + b_ac * AC_AIR
V_CAR = asc_car + b_t * T_CAR_S + b_c * C_CAR
V_NON_MOTOR = asc_non_motor + b_t * T_NON_MOTOR_S

# Aggregated formulation with non-linear perception of travel time
V_RAIL = asc_rail + b_t * T_RAIL + b_t2 * T_RAIL_S + b_c * C_RAIL + b_ac * AC_RAIL
V_COACH = asc_coach + b_t * T_COACH + b_t2 * T_COACH_S + b_c * C_COACH + b_ac * AC_COACH
V_BUS = asc_bus + b_t * T_BUS + b_t2 * T_BUS_S + b_c * C_BUS + b_ac * AC_BUS
V_AIR = asc_air + b_t * T_AIR + b_t2 * T_AIR_S + b_c * C_AIR + b_ac * AC_AIR
V_CAR = asc_car + b_t * T_CAR + b_t2 * T_CAR_S + b_c * C_CAR
V_NON_MOTOR = asc_non_motor + b_t * T_NON_MOTOR + b_t2 * T_NON_MOTOR_S'''
print('Deprected')

Deprected


In [20]:
# The cost damping function
def spline(x, beta=b_gc, Q=3, c0=0, c1=20, c2=40, c3=np.inf):
    alpha = [0, -beta/2*ex.Power(ex.log(c1),3),
             -beta/2*ex.log(c1)*(3*ex.Power(ex.log(c2),2)+ex.Power(ex.log(c1),2))] # for Q=3
    theta = [1, 3/2*ex.log(c1), 3*ex.log(c1)*ex.log(c2)] # for Q=3
    def component(x, q):
        return beta*theta[q-1]*ex.Power(ex.log(x),Q-q+1) + alpha[q-1]
    if x < c1:
        return component(x,1)
    elif x < c2:
        return component(x,2)
    elif x >= c2:
        return component(x,3)

In [21]:
# Aggregated formulation with damped generalised cost
V_RAIL = asc_rail + spline(GC_RAIL)
V_RAIL_S = asc_rail_s + spline(GC_RAIL_S)# + b_ac * AC_RAIL
V_RAIL_L = asc_rail_l + spline(GC_RAIL_L)# + b_ac * AC_RAIL
V_COACH = asc_coach + spline(GC_COACH)# + b_ac * AC_COACH
V_BUS = asc_bus + spline(GC_BUS)# + b_ac * AC_BUS
V_ROAD = asc_road + spline(GC_ROAD)
V_AIR = asc_air + spline(GC_AIR)# + b_ac * AC_AIR
V_CAR = asc_car + spline(GC_CAR)
V_NON_MOTOR = asc_non_motor + spline(GC_NON_MOTOR)

### Run the estimation

In [22]:
# Define level of verbosity
logger = message.bioMessage()
#logger.setSilent()
logger.setWarning()
#logger.setGeneral()
#logger.setDetailed()

In [23]:
# Map modes to utility functions
V = {1:V_RAIL,
#     1:V_RAIL_S,
#     2:V_RAIL_L,
#     3:V_COACH,
     4:V_ROAD,#BUS,
     5:V_AIR,
     6:V_CAR,
     7:V_NON_MOTOR}

In [24]:
# Map the availability of alternatives with MODE as key
# PT is always available
av = {1:RAIL_AV,#RAIL_S_AV,
#      2:RAIL_L_AV,
#      3:COACH_AV,
      4:ROAD_AV,#BUS_AV,
      5:AIR_AV,
      6:CAR_AV,
      7:NON_MOTOR_AV}

In [25]:
# Mode nests as tuples with nest name and dictionary where
# alternative IDs are mapped to alpha values. Missing ID's alpha is zero
# Alternatively use lists with mode ID without alpha
pt = mu_pt, [1, 4]#[1, 2, 3, 4]
air = 1, [5]
car = 1, [6]
walk = 1, [7]
nests = pt, air, car, walk

In [26]:
# Write results to an Excel file
writer = pd.ExcelWriter(input_path + 'estimation_results.xls', engine='xlsxwriter')

In [27]:
# Choose the multinomial logit model
mnl = models.loglogit(V, av, MODE)

In [28]:
model_mnl = bio.BIOGEME(database, mnl)
model_mnl.modelName = 'MNL'

In [29]:
results_mnl = model_mnl.estimate()

In [30]:
results = results_mnl.getEstimatedParameters()
for key, val in results_mnl.getGeneralStatistics().items():
    results.loc[key] = [val[0], val[1]] + ['' for i in range(len(results.columns)-2)]
results

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
asc_air,-6.956952,0.178508,-38.9728,0,0.210644,-33.027,0
asc_non_motor,-4.307181,0.035462,-121.459,0,0.0352195,-122.295,0
asc_rail,-2.186767,0.0139795,-156.426,0,0.0159399,-137.188,0
asc_road,-2.998858,0.0181215,-165.486,0,0.0189347,-158.379,0
b_gc,-0.364716,0.00637818,-57.1819,0,0.013435,-27.1467,0
Number of estimated parameters,5.000000,,,,,,
Sample size,94107.000000,,,,,,
Excluded observations,0.000000,,,,,,
Init log likelihood,-129013.081663,.7g,,,,,
Final log likelihood,-38976.222099,.7g,,,,,


In [31]:
results.to_excel(writer, sheet_name=model_mnl.modelName)

In [32]:
# Choose the logarithmic nested logit model
nl = models.lognested(V, av, nests, MODE)

In [33]:
# Nested Logit
model_nl = bio.BIOGEME(database, nl)
model_nl.modelName = 'NL'

In [34]:
results_nl = model_nl.estimate()

In [35]:
results = results_nl.getEstimatedParameters()
for key, val in results_nl.getGeneralStatistics().items():
    results.loc[key] = [val[0], val[1]] + ['' for i in range(len(results.columns)-2)]
results

,Value,Active bound,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
asc_air,-6.961273,0,0.973822,-7.1484,8.77964e-13,31.9696,-0.217747,0.827626
asc_non_motor,-4.307126,0,0.140344,-30.6898,0,4.53161,-0.950462,0.341877
asc_rail,-2.186758,0,0.147538,-14.8217,0,4.9101,-0.445359,0.65606
asc_road,-2.998901,0,0.587184,-5.10726,3.26864e-07,19.6138,-0.152897,0.878479
b_gc,-0.364742,0,0.1211,-3.0119,0.00259616,4.03755,-0.0903374,0.928019
mu_pt,1.000000,1,0.632113,1.582,0.11365,21.124,0.0473394,0.962243
Number of estimated parameters,6.000000,,,,,,,
Sample size,94107.000000,,,,,,,
Excluded observations,0.000000,,,,,,,
Init log likelihood,-129013.081663,.7g,,,,,,


In [36]:
results.to_excel(writer, sheet_name=model_nl.modelName)

In [37]:
# Run the estimation by purpose
results = []
for p in [1,2,3,4,6]:
    database = db.Database('MiD2017', df.copy())
    database.remove(PURPOSE!=p)
    print('Sample size for purpose {}: {}'.format(p, database.getSampleSize()))
    model = bio.BIOGEME(database, mnl) # Choose the model formulation
    model.modelName = 'MNL_Fz' + str(p) # Name it
    results.append(model.estimate()) # Estimation
    output = results[-1].getEstimatedParameters()
    # Add results to the Excel file
    for key, val in results[-1].getGeneralStatistics().items():
        output.loc[key] = [val[0], val[1]] + ['' for i in range(len(output.columns)-2)]
    output.to_excel(writer, sheet_name=model.modelName)

Sample size for purpose 1: 25368
Sample size for purpose 2: 3671
Sample size for purpose 3: 23062
Sample size for purpose 4: 5624
Sample size for purpose 6: 36382


In [38]:
writer.save()

In [39]:
# Generate LaTeX code
file = open(input_path + 'estimation_results_LaTeX_code.txt', 'w')
for r in results:
    file.write(r.getLaTeX())
file.close()